# Keras Traffic Classification to TensorRT Pipeline

This notebook reproduces the data preprocessing from `traffic_classification.ipynb`, trains a TensorFlow/Keras 2D CNN, and exports the model to ONNX and TensorRT.

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt


2025-09-27 18:31:52.373171: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-27 18:31:52.399263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-27 18:31:52.399284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-27 18:31:52.400028: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-27 18:31:52.405029: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_path = "Data_for_ML_training_Bidirectional"

def load_and_label(pattern, label):
    files = glob(os.path.join(data_path, pattern))
    dfs = [pd.read_csv(f) for f in files]
    df = pd.concat(dfs, ignore_index=True)
    df["binary_label"] = label
    return df

mmtc_df = load_and_label("parsed_mmtc*.csv", 1)
embb_df = load_and_label("parsed_embb*.csv", 0)
urllc_df = load_and_label("parsed_urllc*.csv", 0)

full_df = pd.concat([mmtc_df, embb_df, urllc_df], ignore_index=True)
full_df = full_df.drop(columns=["amf_ue_ngap_id", "ran_ue_id"])

print("\nLabel distribution:")
print(full_df["binary_label"].value_counts())



Label distribution:
binary_label
1    29473
0    21040
Name: count, dtype: int64


In [3]:
normalized_df = full_df.copy()
feature_cols = [col for col in normalized_df.columns if col != "binary_label"]

scaler = MinMaxScaler()
normalized_df[feature_cols] = scaler.fit_transform(normalized_df[feature_cols])

print(f"Normalized {len(feature_cols)} features using MinMaxScaler")

label_names = {0: "non-mMTC", 1: "mMTC"}
for c in sorted(normalized_df["binary_label"].unique()):
    count = len(normalized_df[normalized_df["binary_label"] == c])
    print(f"Class {label_names[c]} = {count} samples")


Normalized 7 features using MinMaxScaler
Class non-mMTC = 21040 samples
Class mMTC = 29473 samples


In [4]:
def generate_binary_slices(df, label_column="binary_label", slice_length=4, step=4, zero_threshold=10):
    """
    Generate time-sliced data from a normalized binary-class dataframe.

    Returns:
        X: (N, slice_length, F) shaped input data.
        y: (N,) labels corresponding to majority class in slice.
    """
    feature_columns = df.columns.drop(label_column)
    data = df[feature_columns].values
    labels = df[label_column].values

    X_slices = []
    y_slices = []
    for i in range(0, len(df) - slice_length + 1, step):
        X_slice = data[i:i+slice_length]
        y_slice = labels[i:i+slice_length]

        # if check_slice_zero_rows(X_slice, zero_threshold):
        #     continue

        majority_label = int(round(np.mean(y_slice)))
        X_slices.append(X_slice)
        y_slices.append(majority_label)
    X_slices = np.array(X_slices)
    y_slices = np.array(y_slices)

    print(f"Generated {X_slices.shape[0]} slices of shape {X_slices.shape[1:]}")

    unique, counts = np.unique(y_slices, return_counts=True)
    for u, c in zip(unique, counts):
        print(f"  Class {u} → {c} samples")

    return X_slices, y_slices


In [5]:
X, y = generate_binary_slices(normalized_df, slice_length=2, step=2, zero_threshold=65)
slice_length = X.shape[1]
feature_count = X.shape[2]

X_flat = X.reshape(X.shape[0], -1)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(sss.split(X_flat, y))

X_train = X[train_idx]
X_test = X[test_idx]
y_train = y[train_idx]
y_test = y[test_idx]

print(f"\nTraining set: {len(X_train):,} samples")
print(f"Test set: {len(X_test):,} samples")
print(f"Train class distribution: {np.bincount(y_train)}")
print(f"Test class distribution: {np.bincount(y_test)}")


Generated 25256 slices of shape (2, 7)
  Class 0 → 10520 samples
  Class 1 → 14736 samples

Training set: 20,204 samples
Test set: 5,052 samples
Train class distribution: [ 8416 11788]
Test class distribution: [2104 2948]


In [6]:
X_train_keras = X_train[:, :, :, np.newaxis].astype("float32")
X_test_keras = X_test[:, :, :, np.newaxis].astype("float32")
y_train_float = y_train.astype("float32")
y_test_float = y_test.astype("float32")

input_shape = (slice_length, feature_count, 1)

print("Keras input shape:", input_shape)
print("X_train_keras shape:", X_train_keras.shape)
print("X_test_keras shape:", X_test_keras.shape)


Keras input shape: (2, 7, 1)
X_train_keras shape: (20204, 2, 7, 1)
X_test_keras shape: (5052, 2, 7, 1)


In [7]:
from tensorflow.keras import layers, models, callbacks

model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(1, 2)),
    layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(1, 2)),
    layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 7, 32)          320       
                                                                 
 batch_normalization (Batch  (None, 2, 7, 32)          128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 3, 32)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 3, 64)          18496     
                                                                 
 batch_normalization_1 (Bat  (None, 2, 3, 64)          256       
 chNormalization)                                                
                                                        

2025-09-27 18:31:53.982409: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
early_stop = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True,
)

history = model.fit(
    X_train_keras,
    y_train_float,
    validation_data=(X_test_keras, y_test_float),
    epochs=60,
    batch_size=64,
    callbacks=[early_stop],
    verbose=2,
)


Epoch 1/60
316/316 - 2s - loss: 0.4003 - accuracy: 0.7932 - val_loss: 0.6665 - val_accuracy: 0.6799 - 2s/epoch - 7ms/step
Epoch 2/60
316/316 - 1s - loss: 0.3588 - accuracy: 0.8100 - val_loss: 0.8299 - val_accuracy: 0.4165 - 1s/epoch - 4ms/step
Epoch 3/60
316/316 - 1s - loss: 0.3507 - accuracy: 0.8120 - val_loss: 1.3332 - val_accuracy: 0.4153 - 1s/epoch - 4ms/step
Epoch 4/60
316/316 - 1s - loss: 0.3475 - accuracy: 0.8126 - val_loss: 0.8307 - val_accuracy: 0.4165 - 1s/epoch - 4ms/step
Epoch 5/60
316/316 - 1s - loss: 0.3455 - accuracy: 0.8145 - val_loss: 1.2389 - val_accuracy: 0.4285 - 1s/epoch - 4ms/step
Epoch 6/60
316/316 - 1s - loss: 0.3438 - accuracy: 0.8142 - val_loss: 1.6031 - val_accuracy: 0.4165 - 1s/epoch - 4ms/step
Epoch 7/60
316/316 - 1s - loss: 0.3462 - accuracy: 0.8139 - val_loss: 2.0907 - val_accuracy: 0.6793 - 1s/epoch - 4ms/step
Epoch 8/60
316/316 - 1s - loss: 0.3424 - accuracy: 0.8156 - val_loss: 1.0268 - val_accuracy: 0.4165 - 1s/epoch - 4ms/step
Epoch 9/60
316/316 - 1s 

In [9]:
eval_loss, eval_acc = model.evaluate(X_test_keras, y_test_float, verbose=0)
print(f"Test Loss: {eval_loss:.4f}")
print(f"Test Accuracy: {eval_acc:.4f}")

y_pred_probs = model.predict(X_test_keras).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["non-mMTC", "mMTC"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Loss: 0.6665
Test Accuracy: 0.6799
158/158 [==============================] - 0s 1ms/step

Classification Report:
              precision    recall  f1-score   support

    non-mMTC       0.96      0.24      0.39      2104
        mMTC       0.65      0.99      0.78      2948

    accuracy                           0.68      5052
   macro avg       0.80      0.62      0.58      5052
weighted avg       0.78      0.68      0.62      5052

Confusion Matrix:
[[ 507 1597]
 [  20 2928]]


In [10]:
h5_path = "traffic_cnn_keras.h5"
model.save(h5_path)
print("Saved Keras model to", h5_path)


Saved Keras model to traffic_cnn_keras.h5


/home/vs24436/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
import tf2onnx

onnx_path = "traffic_cnn_keras.onnx"

input_signature = (
    tf.TensorSpec(
        [None, slice_length, feature_count, 1],
        tf.float32,
        name=model.inputs[0].name.split(":")[0],
    ),
)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,
)

with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print("Saved ONNX model to", onnx_path)


Saved ONNX model to traffic_cnn_keras.onnx


2025-09-27 18:32:07.012406: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2025-09-27 18:32:07.012480: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-09-27 18:32:07.138071: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-09-27 18:32:07.184888: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2025-09-27 18:32:07.184986: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-09-27 18:32:07.185410: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GP

In [13]:
import tensorrt as trt

trt_logger = trt.Logger(trt.Logger.WARNING)

def build_trt_engine(onnx_file_path, engine_file_path, max_ws=1 << 30, fp16=True):
    with trt.Builder(trt_logger) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, trt_logger) as parser:
        config = builder.create_builder_config()
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, max_ws)
        if fp16 and builder.platform_has_fast_fp16:
            config.set_flag(trt.BuilderFlag.FP16)

        with open(onnx_file_path, "rb") as f:
            if not parser.parse(f.read()):
                for idx in range(parser.num_errors):
                    print(parser.get_error(idx))
                raise RuntimeError("Failed to parse ONNX model")

        input_tensor = network.get_input(0)
        min_shape = (1, slice_length, feature_count, 1)
        opt_batch = min(16, max(1, X_train_keras.shape[0]))
        max_batch = max(opt_batch, 32)
        opt_shape = (opt_batch, slice_length, feature_count, 1)
        max_shape = (max_batch, slice_length, feature_count, 1)

        profile = builder.create_optimization_profile()
        profile.set_shape(input_tensor.name, min_shape, opt_shape, max_shape)
        config.add_optimization_profile(profile)

        engine_bytes = builder.build_serialized_network(network, config)
        if engine_bytes is None:
            raise RuntimeError("Engine build failed")

        with open(engine_file_path, "wb") as f:
            f.write(engine_bytes)
        print("Saved TensorRT engine to", engine_file_path)

trt_engine_path = "traffic_cnn_keras.trt"
build_trt_engine(onnx_path, trt_engine_path, fp16=True)


Saved TensorRT engine to traffic_cnn_keras.trt
